# Thematic Stock Screening with everyrow.io/screen

Screen the S&P 500 for a combined qualitative investment thesis:
1. **Recurring revenue >75%** - high-quality subscription/contract-based businesses
2. **Taiwan tensions beneficiary** - companies positioned to benefit from US-China geopolitical tensions

This is a filter that no traditional stock screener can perform.

In [ ]:
# Setup
import asyncio
from pathlib import Path

import pandas as pd
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Load API key from .env
load_dotenv()

from everyrow.ops import screen

In [2]:
# Load S&P 500 companies
stocks = pd.read_csv("../data/S&P 500 Companies.csv")
print(f"Loaded {len(stocks)} companies")
stocks.head()

Loaded 502 companies


,cik,company,ticker,market_cap_usd_billion,gics_sector,gics_sub_industry,headquarters,founded
0,1045810,Nvidia,NVDA,4425.366110,Information Technology,Semiconductors,"Santa Clara, California",1993
1,320193,Apple Inc.,AAPL,4247.171105,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1977
2,1652044,Alphabet Inc.(Class A),GOOGL,3825.129226,Communication Services,Interactive Media & Services,"Mountain View, California",1998
3,1652044,Alphabet Inc.(Class C),GOOG,3814.940738,Communication Services,Interactive Media & Services,"Mountain View, California",1998
4,789019,Microsoft,MSFT,3642.251543,Information Technology,Systems Software,"Redmond, Washington",1975


In [3]:
# Define the screening criteria
SCREENING_TASK = """
Find companies with high-quality recurring revenue business models that would
also benefit from escalating US-China tensions over Taiwan.

**Recurring revenue >75%**: Subscription services, long-term contracts,
maintenance agreements, royalty streams. Not one-time product sales or
project-based work. Be conservative when estimating.

**Taiwan tensions beneficiary**: Companies that would see increased revenue
or strategic importance from Taiwan tensions - think CHIPS Act beneficiaries,
defense contractors, cybersecurity, reshoring plays, alternative supply chain
providers. Exclude companies dependent on Taiwan manufacturing or with
significant China revenue at risk.
"""

# Output schema - just pass/fail for efficiency
class ScreenResult(BaseModel):
    passes: bool = Field(
        description="True if company has >75% recurring revenue AND is a Taiwan tensions beneficiary"
    )

In [ ]:
# Run the screen
async def run_screen():
    print("Screening... (this will take a few minutes)\n")
    
    result = await screen(
        task=SCREENING_TASK,
        input=stocks,
        response_model=ScreenResult,
    )
    
    return result.data

# Run it
results = await run_screen()

In [5]:
# Summary
print(f"Companies passing both criteria: {len(results)}")
print(f"Pass rate: {len(results)/len(stocks)*100:.1f}% of S&P 500\n")

print("Passing companies:")
for _, row in results.iterrows():
    print(f"  {row['ticker']:6} | {row['company'][:40]}")

Companies passing both criteria: 63
Pass rate: 12.5% of S&P 500

Passing companies:
  ORCL   | Oracle Corporation
  PLTR   | Palantir Technologies
  TMUS   | T-Mobile US
  CRM    | Salesforce
  BX     | Blackstone Inc.
  NEE    | NextEra Energy
  NOW    | ServiceNow
  SPGI   | S&P Global
  PANW   | Palo Alto Networks
  CRWD   | CrowdStrike
  PLD    | Prologis
  CEG    | Constellation Energy
  KKR    | KKR & Co.
  PH     | Parker Hannifin
  ADP    | Automatic Data Processing
  CVS    | CVS Health
  SO     | Southern Company
  DUK    | Duke Energy
  GD     | General Dynamics
  MMC    | Marsh McLennan
  WM     | Waste Management
  BK     | BNY Mellon
  NOC    | Northrop Grumman
  TDG    | TransDigm Group
  AON    | Aon plc
  CTAS   | Cintas
  EQIX   | Equinix
  ADSK   | Autodesk
  AJG    | Arthur J. Gallagher & Co.
  KMI    | Kinder Morgan
  WDAY   | Workday, Inc.
  DDOG   | Datadog
  DLR    | Digital Realty
  ROP    | Roper Technologies
  XEL    | Xcel Energy
  AXON   | Axon Enterprise
 

In [6]:
# View the research for a specific company
if 'research' in results.columns:
    sample = results[results['ticker'] == 'NOW'].iloc[0]
    print(f"Research for {sample['company']}:\n")
    print(sample['research'])

Research for ServiceNow:

{'screening_result': 'ServiceNow has extremely high recurring revenue, with approximately 97% of its total revenue coming from subscriptions. It is a beneficiary of US-China tensions as its platform is critical for digital transformation, cybersecurity, and US federal government infrastructure; its public sector business grew 30% in early 2025. It has low exposure to China (Asia Pacific accounts for only ~11% of revenue total).', 'passes': 'ServiceNow has extremely high recurring revenue, with approximately 97% of its total revenue coming from subscriptions. It is a beneficiary of US-China tensions as its platform is critical for digital transformation, cybersecurity, and US federal government infrastructure; its public sector business grew 30% in early 2025. It has low exposure to China (Asia Pacific accounts for only ~11% of revenue total).'}


In [7]:
# Breakdown by sector
if 'gics_sector' in results.columns:
    print("Passing companies by sector:")
    print(results['gics_sector'].value_counts())

Passing companies by sector:
gics_sector
Information Technology    13
Utilities                 13
Financials                12
Industrials               11
Real Estate                5
Energy                     4
Health Care                3
Communication Services     1
Materials                  1
Name: count, dtype: int64


In [8]:
# Save results
results.to_csv("thematic_screen_results.csv", index=False)
print(f"Results saved to thematic_screen_results.csv")

Results saved to thematic_screen_results.csv


## What Just Happened?

The everyrow.io screen:

1. **Researched each company** using web search to understand their business model
2. **Evaluated recurring revenue** by finding actual revenue breakdowns in 10-Ks and earnings
3. **Assessed geopolitical positioning** by researching supply chains, government contracts, and China exposure
4. **Made a judgment** on whether both criteria were met

This would take an analyst weeks to do manually for 500 companies.

## Try Your Own Thesis

Just change `SCREENING_TASK` to any criteria you can describe in plain English:

- "Companies with founder still as CEO"
- "Companies that would benefit from AI infrastructure buildout"
- "Companies with >50% international revenue but <5% China exposure"

If you can describe it, everyrow can screen for it.